# Import functions

In [3]:
#import WAPORWA modules_______________________________________________________________________________________
import os
os.chdir(r'C:\Users\h.jayasekara\Downloads\modules\modules') #change to modules path
# ____________________________________________________________________________________________________________


import glob
import WaPOR
import WA
from WA.pickle_basin import pickle_in,pickle_out  
import pandas as pd
from osgeo import gdal
import numpy as np
import osr
from WaPOR import GIS_functions as gis
import calendar
import sys
import geopandas as gpd
import rasterio
import rasterio.mask
from rasterstats import zonal_stats
import json


# f931d000c500d538478b92992be10702c08e8656b6b54d43d8eecfad9ee8edeee4b34903360c03ac


Your WaPOR API Token: f931d000c500d538478b92992be10702c08e8656b6b54d43d8eecfad9ee8edeee4b34903360c03ac


# Collect data from WAPOR

In [ ]:
WaPOR.API.version=2
catalog=WaPOR.API.getCatalog()

INPUT_FOLDER=r'C:\Users\h.jayasekara\Desktop\WaporData' #change to input path

#%% DEFINE Basin extent
### OPTION 1: Define minimum and maximum longitude (x) and latitude (y)
# xmin,ymin,xmax,ymax=[34.4,29.4,37.0,33.7]

### OPTION 2: Read from shapefile
BASIN_SHP = r'C:\Users\h.jayasekara\Downloads\Sri Lanka SHP\Provinces\Provinces_SL.shp' #path to Basin shapefile
import shapefile
shape=shapefile.Reader(BASIN_SHP)
xmin,ymin,xmax,ymax=shape.bbox

start_date='2022-03-01'
end_date='2022-12-31'

# WaPOR.PCP_daily(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25])

# WaPOR.PCP_monthly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.25,ymax+0.25],
#                   lonlim=[xmin-0.25,xmax+0.25])

# WaPOR.RET_monthly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.5,ymax+0.5],
#                   lonlim=[xmin-0.5,xmax+0.5])

# WaPOR.LCC_yearly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

# WaPOR.I_dekadal(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin,ymax],
#                   lonlim=[xmin,xmax],level=2)

# WaPOR.AET_monthly(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.5,ymax+0.5],
#                   lonlim=[xmin-0.5,xmax+0.5],level=2)

WaPOR.NPP_monthly(INPUT_FOLDER,Startdate=start_date,
                  Enddate=end_date,latlim=[ymin-0.5,ymax+0.5],
                  lonlim=[xmin-0.5,xmax+0.5],level=2)

# WaPOR.E_dekadal(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.5,ymax+0.5],
#                   lonlim=[xmin-0.5,xmax+0.5],level=2)

# WaPOR.T_dekadal(INPUT_FOLDER,Startdate=start_date,
#                   Enddate=end_date,latlim=[ymin-0.5,ymax+0.5],
#                   lonlim=[xmin-0.5,xmax+0.5],level=2)

# Create Biomass ( using NPP data ) or ET for the Maha season

In [5]:
# user inputs ________________________________________________________________________________________________

Monthly_product_folder=r"C:\Users\h.jayasekara\Desktop\WaporData\L2_NPP_M" #insert file path
ds_code='L2_NPP_M'    #ds code for file naming should insetr refering WaPOR docs
shapefile = r"C:\Users\h.jayasekara\Desktop\WaporData\Sri_lanka\final1.shp"     #path_to_shapefile.shp
start_date='2015-01-01'
end_date='2022-12-31'
season = "Maha"  # select the season  -->  "Maha"  "Yala"
# ____________________________________________________________________________________________________________


product_name = ds_code.split("_")
naming_product = product_name[1]

path_split = os.path.split(Monthly_product_folder)
ds_code_f = path_split[1]

#output folder name
# output_folder=Monthly_product_folder.replace(ds_code_f,'Maha_{}'.format(naming_product)) 

# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)
    

if (naming_product == "NPP"):
    
    Biomass_out = Monthly_product_folder.replace(ds_code_f,'Maha_{}_to_Biomass'.format(naming_product)) 
    if not os.path.exists(Biomass_out):
        os.makedirs(Biomass_out)
        
    res_product = "biomass"
    
elif (naming_product == "AETI"):
    
    ET_out = Monthly_product_folder.replace(ds_code_f,'Maha_{}'.format(naming_product)) 
    if not os.path.exists(ET_out):
        os.makedirs(ET_out)
        
    res_product = "AETI"
        
else:
    print("error in ds_code you entered")
    sys.exit()

# ____________________________________________________________________________________________________________
    
    
input_fhs=glob.glob(os.path.join(Monthly_product_folder,'*.tif'))
input_fhs

#Get year to array
year_dates=pd.date_range(start_date,end_date,freq='Y')



print ("finished")

Loading WaPOR catalog...
finished


In [7]:
# # create the sum image _____________________________________________________________________________________
#Get df avail 
WaPOR.API.version=2
df_avail=WaPOR.API.getAvailData(ds_code,time_range='{0},{1}'.format(start_date,end_date))


driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(input_fhs[0])
sumArrays_Jan_Mar_ = {}
sumArrays_Nov_Dec_ = {}

# for creating sumArray for Jan to Mar


for date in year_dates:   
    year_fhs=[]
    
    for fh in input_fhs:
        raster_id=os.path.split(fh)[-1].split('.tif')[0][-7:]
        year=int(raster_id[0:4])
        month=int(raster_id[5:7])
        if (year == date.year) &  (month == 1 or month == 2 or month == 3):
            year_fhs.append(fh)
            
    SumArray=np.zeros((ysize,xsize),dtype=np.float32)
    for f in year_fhs:
        Array=gis.OpenAsArray(f,nan_values=True)
        SumArray+=Array
        
    sumArrays_Jan_Mar_[date.year] = SumArray
     

# for creating sumArray for Nov to Dec  

for date in year_dates:
    year_fhs=[]
    
    for fh in input_fhs:
        raster_id=os.path.split(fh)[-1].split('.tif')[0][-7:]
        year=int(raster_id[0:4])
        month=int(raster_id[5:7])
        if (year == date.year) &  (month == 11 or month == 12):
            year_fhs.append(fh)
            
    SumArray=np.zeros((ysize,xsize),dtype=np.float32)
    for f in year_fhs:
        Array=gis.OpenAsArray(f,nan_values=True)
        SumArray+=Array
        
    sumArrays_Nov_Dec_[date.year] = SumArray
    
    
# create the final array & save it ___________________________________________________________________________      

    
output_folder_clip=Monthly_product_folder.replace(ds_code_f,'{}_{}_Clipped'.format(season,res_product))
if not os.path.exists(output_folder_clip):
    os.makedirs(output_folder_clip)

# read shp file
gdf = gpd.read_file(shapefile)  
    
for date in year_dates:

    if date.year not in sumArrays_Nov_Dec_:
        print("Array for year {} not found in sumArrays_Nov_Dec_".format(date.year))
        continue
        
    if (date.year+1) not in sumArrays_Jan_Mar_:
        print("Array for year {} not found in sumArrays_Jan_Mar_".format(date.year+1))
        continue
        
    Maha_sumArray = sumArrays_Jan_Mar_[date.year+1] + sumArrays_Nov_Dec_[date.year]
    

    Maha_sumArray_c =  Maha_sumArray
    
    
    if (naming_product == "NPP"): 
        
        biomass = Maha_sumArray_c * 88.888 / 2.55

        # Save the biomass as a GeoTIFF file
        out_fh = os.path.join(Biomass_out, 'WaPOR_{}_Biomass_{}.tif'.format(season,date.year))  
        gis.CreateGeoTiff(out_fh, biomass, driver, NDV, xsize, ysize, GeoT, Projection)
    
    elif (naming_product == "AETI"):
        
        # Save the product as a GeoTIFF file
        out_fh = os.path.join(ET_out, 'WaPOR_{}_ET_{}.tif'.format(season,date.year))  
        gis.CreateGeoTiff(out_fh, Maha_sumArray_c, driver, NDV, xsize, ysize, GeoT, Projection)
        
#     with rasterio.open(out_fh) as src:
#         out_image, out_transform = rasterio.mask.mask(src, gdf.geometry, crop=True)
#         out_meta = src.meta.copy()

#     out_meta.update({"driver": "GTiff",
#                      "height": out_image.shape[1],
#                      "width": out_image.shape[2],
#                      "transform": out_transform})

#     with rasterio.open(os.path.join(ET_out, 'WaPOR_ET_{}_clipped.tif'.format(date.year)), "w", **out_meta) as dest:
#         dest.write(out_image)
        
    #clipping the product created ____________________________________________________________________________ 
    
    with rasterio.open(out_fh) as src:
        out_image, out_transform = rasterio.mask.mask(src, gdf.geometry, crop=True)
        out_meta = src.meta.copy()

    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform})

    with rasterio.open(os.path.join(output_folder_clip, 'WaPOR_{}_{}.tif'.format(res_product,date.year)), "w", **out_meta) as dest:
        dest.write(out_image)

    
# ____________________________________________________________________________________________________________
# if it is for a year summation
# for date in year_dates:
        
#     year_fhs=[]
# #     SumArray = np.zeros((ysize, xsize), dtype=np.float32)
#     for fh in input_fhs:
#         raster_id=os.path.split(fh)[-1].split('.tif')[0][-7:]
#         year=int(raster_id[0:4])
#         month=int(raster_id[5:7])
#         if (year == date.year):
#             year_fhs.append(fh)
         
#     SumArray=np.zeros((ysize,xsize),dtype=np.float32)
#     for f in year_fhs:
#         Array=gis.OpenAsArray(f,nan_values=True)
#         SumArray+=Array
        
    
#     SumArray = correction_factor * SumArray

#     # Save the result as a GeoTIFF file
#     out_fh = os.path.join(output_folder, 'WaPOR_{}_{}.tif'.format(naming_product,date.year))  
#     gis.CreateGeoTiff(out_fh, SumArray, driver, NDV, xsize, ysize, GeoT, Projection)

Array for year 2023 not found in sumArrays_Jan_Mar_


## Calculatiting zonal statics

In [33]:
import geopandas as gpd
from shapely.geometry import mapping
import fiona
#  User inputs _______________________________________________________________________________________________

res_product = "water_productivity" # select the product  -->  "AETI"  "Biomass" "water_productivity"
season = "Maha"  # select the season  -->  "Maha"  "Yala"
# ____________________________________________________________________________________________________________

if (res_product == "water_productivity"):
    req_folder = "Water_productivity"
    req_file = "Water_productivity"
    res_file = "Water_productivity"

    
if (res_product == "AETI"):
    req_folder = "Maha_AETI_Clipped"
    req_file = "WaPOR_AETI"
    res_file = "AETI"

    
if (res_product == "Biomass"):
    req_folder = "Maha_biomass_Clipped"
    req_file = "WaPOR_biomass"
    res_file = "Biomass"


polygon =  r"C:\Users\h.jayasekara\Desktop\admin\lka_admbnda_adm1_slsd_20200305.shp"
# polygon =  r"C:\Users\h.jayasekara\Desktop\admin\lka_admbnda_adm2_slsd_20200305.shp"
# polygon =  r"C:\Users\h.jayasekara\Desktop\admin\lka_admbnda_adm3_slsd_20200305.shp"
# polygon =  r"C:\Users\h.jayasekara\Desktop\admin\lka_admbnda_adm4_slsd_20200305.shp"

polygon_split = os.path.split(polygon)[-1].split(".shp")
file_name = polygon_split[0]
file_split = file_name.split("_")
admin_level = file_split[2]


if (admin_level == "adm1"):
    OID = 'ADM1_PCODE'

elif (admin_level == "adm2"):
    OID = 'ADM2_PCODE'
    

elif (admin_level == "adm3"):
    OID = 'ADM3_PCODE'
    
elif (admin_level == "adm4"):
    OID = 'ADM4_PCODE'
    
else:
    print("It seems the file name is not in the format used in this code")
    sys.exit()
       
gdf = gpd.read_file(polygon)

zonal_statistics_province={}
zonal_statistics_district={}
zonal_statistics_DSD={}
zonal_statistics_GND={}

if not os.path.isdir(r"C:\Users\h.jayasekara\Desktop\WaporData\JSON_1"):
    os.makedirs(r"C:\Users\h.jayasekara\Desktop\WaporData\JSON_1")

for date in year_dates:

    raster_1 = r"C:\Users\h.jayasekara\Desktop\WaporData\{}\{}_{}.tif".format(req_folder,req_file,date.year)
    if not os.path.isfile(raster_1):
        print("raster for year {} not found in the dataset".format(date.year))
        continue
        
    raster_r = rasterio.open(raster_1)
    array = raster_r.read(1)
    affine = raster_r.transform

    stat = zonal_stats(gdf, array ,affine = affine, stats= 'mean', nodata=-9999.0, geojson_out=True )
    #  must be one of  ['count', 'min', 'max', 'mean', 'sum', 'std', 'median', 'majority', 'minority', 'unique', 'range', 'nodata', 'nan']
    # the last nodata=-9999.0 is for exclude nodata values
    
    
    # input id into zonal stat array
#     stat_id = [{**s, 'id': row[1][OID]} for s, row in zip(stat, gdf.iterrows())]

    
    if (admin_level == "adm1"):
        zonal_statistics_province[date.year] = stat
        json_zonal_stat = json.dumps(zonal_statistics_province)


    elif (admin_level == "adm2"):
        zonal_statistics_district[date.year] = stat
        json_zonal_stat = json.dumps(zonal_statistics_district)
      

    elif (admin_level == "adm3"):
        zonal_statistics_DSD[date.year] = stat
        json_zonal_stat = json.dumps(zonal_statistics_DSD)

        
        
    elif (admin_level == "adm4"):
        zonal_statistics_GND[date.year] = stat
        json_zonal_stat = json.dumps(zonal_statistics_GND)        


    jsonFile = open(r"C:\Users\h.jayasekara\Desktop\WaporData\JSON_1\{}_{}_{}.geojson".format(res_file,admin_level,date.year), "w")
    jsonFile.write(json_zonal_stat)
    jsonFile.close()
# print("finished")

raster for year 2022 not found in the dataset


# JSON to csv

In [116]:
import pandas as pd

if not os.path.isdir(r"C:\Users\h.jayasekara\Desktop\WaporData\csv"):
    os.makedirs(r"C:\Users\h.jayasekara\Desktop\WaporData\csv")

directory_path = r"C:\Users\h.jayasekara\Desktop\WaporData\JSON"
files = os.listdir(directory_path)
i=0

while i<len(files):
    
    file_path = r"C:\Users\h.jayasekara\Desktop\WaporData\JSON\{}".format(files[i])
    file_split = os.path.split(file_path)[-1].split(".json")
    file_name = file_split[0]
    file_split = file_name.split("_")
    admin_level = file_split[-1]

    file = open(file_path)
    data = json.load(file)




    df = pd.DataFrame()
    csv_file = r"C:\Users\h.jayasekara\Desktop\WaporData\csv\{}.csv".format(file_name)
    for year, values in data.items():
        # Create a dataframe for the year's data
        year_df = pd.DataFrame(values)
        # Set the index of the year's dataframe to the "id" column
        year_df.set_index("id", inplace=True)
        # Rename the "mean" column to the year
        year_df.rename(columns={"mean": "mean_{}".format(year)}, inplace=True)
#         year_df.rename(columns={"id":admin_level }, inplace=True)
        # Append the year's dataframe to the main dataframe
        df = pd.concat([df, year_df], axis=1)           


    # save the dataframe to a CSV file
    df.to_csv(csv_file, index=True)          

    i = i+1


# Water Productivity 

In [8]:
Water_Productivity = Monthly_product_folder.replace(ds_code_f,'Water_productivity') 
if not os.path.exists(Water_Productivity):
    os.makedirs(Water_Productivity)
    
for date in year_dates:    
    try:
        src1 = rasterio.open(r"C:\Users\h.jayasekara\Desktop\WaporData\Maha_Biomass_Clipped\WaPOR_Biomass_{}.tif".format(date.year))
        src2 = rasterio.open(r"C:\Users\h.jayasekara\Desktop\WaporData\Maha_AETI_Clipped\WaPOR_AETI_{}.tif".format(date.year))

        data1 = src1.read(1)
        data2 = src2.read(1)

        mask = data2 == 0
        
        result = np.zeros_like(data1, dtype=np.float32)
        result[~mask] = np.divide(data1[~mask], data2[~mask])

        nodata1 = src1.nodata
        nodata2 = src2.nodata

        if nodata1 is not None and nodata2 is not None:
            nodata_mask = (data1 == nodata1) | (data2 == nodata2)
            result[nodata_mask] = nodata1

        meta = src1.meta
        meta.update(dtype=result.dtype, count=1)

        with rasterio.open(r"C:\Users\h.jayasekara\Desktop\WaporData\Water_productivity\Water_productivity_{}.tif".format(date.year), 'w', **meta) as dst:
            dst.write(result, 1)
            
    except rasterio.RasterioIOError:
        print("File not found for year {}".format(date.year))

File not found for year 2022
